In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import muscle
import muscle_utils

In [ ]:
## Objective function definition
def objective(Fcemax, Lmax, Lce0, Lts, Spe, phi_m, phi_v, alpha, l, v, a, m, y, pso=True):    
    length = l
    vel = v
    
    # CE Element
    # Real-Time Myoprocessors for a Neural Controlled Powered Exoskeleton Arm
    Vcemax = 2 * Lce0 + 8 * Lce0 * alpha
    Vce0 = 1/2 * (a + 1) * Vcemax

    fl = np.exp(-1/2 * ((length/Lce0 - phi_m) / phi_v) ** 2)
    fv = 0.1433 / (0.1074 + np.exp(-1.3 * np.sinh(2.8 * vel / Vce0 + 1.64)))
    Fce = a * fl * fv * Fcemax

    # PE Element
    # Real-Time Myoprocessors for a Neural Controlled Powered Exoskeleton Arm
    Fpemax = 0.05 * Fcemax
    DLpemax = Lmax - (Lce0 + Lts)

    Fpe = Fpemax / (np.exp(Spe) - 1) * (np.exp(Spe/DLpemax * length) - 1)
    
    Ftot = Fce + Fpe
    
    t_out = np.multiply(Ftot,m)
    
    mse = ((y - t_out) ** 2).mean()
    cost = sum(abs(y -t_out))
    if pso:
        return cost
    else:
        return T

In [ ]:
## Generate testdata
m = muscle.Muscle(muscle_type=muscle_utils.MUSCLE_NAME.BICEPS_BRACHII)
a = np.sin(np.linspace(0, 10*np.pi, 140))
a[a < 0] = -a[a < 0]
T = np.zeros((1, 140))
L = np.zeros((1, 140))
M = np.zeros((1, 140))
for i, activation_level in zip(range(0,140), a):
    angles = [0, 0, i, 0]
    M[0, i] = muscle_utils.get_muscle_value(muscle_utils.MUSCLE_NAME.BICEPS_BRACHII, angles[2], muscle_utils.MUSCLE_JOINT.ELBOW)
    T[0, i] = m.get_torque_estimate(angles, activation_level, muscle_utils.MUSCLE_JOINT.ELBOW) * M[0, i]
    L[0, i] = muscle_utils.get_muscle_value(muscle_utils.MUSCLE_NAME.BICEPS_BRACHII, angles)
    
V = np.diff(L)
V = np.insert(V, 0, V[0,0])
L = L - 378.06

In [ ]:
## Initialize PSO
# PSO constants
w = 0.5
phi_p = 0.75
phi_g = 1

# PSO init
S = 100 # particles
n = 8 # dimensions

# Fcemax, Lmax, Lce0, Lts, Spe, phi_m, phi_v, alpha
expected = np.array([462, 404, 131, 230, 9, 0.1, 0.5, 0.56])
min_val  = np.array([300, 100, 100, 100, 0, 0, 0, 0.3])
max_val  = np.array([700, 800, 300, 300, 10, 1, 1, 0.7])
max_vel  = max_val - min_val

In [ ]:
# init
x = np.zeros((S,n))
v = np.zeros((S,n))
p = np.zeros((S,n))
for i in range(S):
    for d in range(n):
        x[i,d] = np.random.uniform(min_val[d], max_val[d], 1)
        v[i,d] = np.random.uniform(-max_vel[d], max_vel[d], 1)

        
for i in range(S):
    for d in range(n):
        p[i,d] = np.random.uniform(min_val[d], max_val[d], 1)
    
gb = max_val
g_best = np.nan
for i in range(S):
    if not (all(min_val <= x[i,:]) and all(x[i,:] <= max_val)):
        continue
        
    tmp = objective(p[i,0], p[i,1], p[i,2], p[i,3], p[i,4], p[i,5], p[i,6], p[i,7], L, V, a, M, T)
    
    if np.isnan(tmp):
        continue
    
    if tmp < g_best or np.isnan(g_best):
        gb = p[i,:]

In [ ]:
# PSO loop
N = 0
N_max = 100
reinit = 0
g_best = objective(gb[0], gb[1], gb[2], gb[3], gb[4], gb[5], gb[6], gb[7], L, V, a, M, T) 
print('INITIAL:', g_best)

while g_best > 100 and N < N_max:
    for i in range(S):
        for d in range(n):
            rp = np.random.rand()
            rg = np.random.rand()
            v[i,d] = w * v[i,d] + phi_p * rp * (p[i,d]-x[i,d]) + phi_g * rg * (gb[d]-x[i,d])
        
        xnew = x[i,:] + v[i,:]
        fitness = objective(xnew[0], xnew[1], xnew[2], xnew[3], xnew[4], xnew[5], xnew[6], xnew[7], L, V, a, M, T)
        p_best = objective(p[i,0], p[i,1], p[i,2], p[i,3], p[i,4], p[i,5], p[i,6], p[i,7], L, V, a, M, T)
        if np.isnan(fitness) or (any(min_val < x[i,:]) or any(x[i,:] > max_val)):
            fitness = np.inf
        
        if fitness < p_best:
            p[i,:] = xnew
            if fitness < g_best:
                gb = xnew
                g_best = fitness

        x[i,:] = xnew
        
    N = N + 1
    if N % int(N_max/10) == 0:
        print(int(N/N_max * 100))
        
print('FINAL:', g_best)

In [ ]:
est = objective(gb[0], gb[1], gb[2], gb[3], gb[4], gb[5], gb[6], gb[7], L, V, a, M, T, pso=False)
plt.plot(T.T)
plt.plot(est.T, 'x')
plt.savefig('pso_esimation.png')

In [ ]:
plt.plot(p[:,0], p[:,1], 'x')
plt.plot(x[:,0], x[:,1], 'x')
plt.plot(gb[0], gb[1], 'o')

In [ ]:
[print(p) for p in gb]

In [ ]:
[print(p) for p in expected]

In [ ]:
any(min_val > gb)

In [ ]:
any(gb > max_val)

In [ ]:
plt.plot(np.log(range(1)))